In [83]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from permetrics.regression import Metrics

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR
from sklearn.linear_model import Ridge, LinearRegression, Lasso, ElasticNet

from tensorflow import keras
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

## Combine all dataframes

In [68]:
# Lead behandling and select relevant columns
behandling = pd.read_csv(Path('../20210324/with_name/behandling_optillfälle.csv'), sep=';')
behandling = behandling[['Der_Behandling_PK',
                         'Der_Opkort_FK',
                         'Der_Anestesikort_FK',
                         'Der_Vårdform_FK',
                         'Der_Prioritet_FK',
                         'BehandlingsStatus',
                         'ASAklass',
                         'ForberedelsetidStartTidpunkt',
                         'ForberedelsetidSlutTidpunkt',
                         'PatientÅlderVidOp',
                         'Veckodag',
                         'Starttimme',
                         'BMI',
                         'Kroppslängd',
                         'Kroppsvikt',
                        ]]
behandling = behandling[behandling['BehandlingsStatus'] == 'Opererad'] # Remove 'abrutna' operationer as they do not contain all relevant data
print("Behandling length: {}".format(len(behandling)))

# Load ingrepp and select relevant columns
ingrepp = pd.read_csv(Path('../20210324/with_name/op_ingrepp_namn.csv'))
ingrepp = ingrepp[['Der_Behandling_PK',
                   'Ingreppkod',
                   'Primär_Sekundär',
                   'Sida',
                  ]]
ingrepp = ingrepp[ingrepp['Primär_Sekundär'] == 'Primär'] # Might want to include this if we make a more complicated model
print("Ingrepp length: {}".format(len(ingrepp)))

# Load diagnos and select relevant columns
diagnos = pd.read_csv(Path('../20210324/with_name/op_diagnos_namn.csv'))
diagnos = diagnos[['Der_Behandling_PK',
                   'Diagnoskod',
                   'Primär_Sekundär',
                  ]]
diagnos = diagnos[diagnos['Primär_Sekundär'] == 'Primär'] # Might want to include this if we make a more complicated model
print("Diagnos length: {}".format(len(diagnos)))

# Combine the data frames
combined_df = behandling.merge(diagnos, on='Der_Behandling_PK').merge(ingrepp, on='Der_Behandling_PK')
print("Combined length: {}".format(len(combined_df)))

# Calculate and add time to the dataframe

# Bad algoritm for checking min and max time of förbereds
start = combined_df["ForberedelsetidStartTidpunkt"].dropna()
slut = combined_df["ForberedelsetidSlutTidpunkt"].dropna()

start_times = []
for time in start:
    minn = int(time[-9:-7])
    hour = int(time[-12:-10])
    minutes = hour*60 + minn
    start_times.append(minutes)
    
stop_times = []
for time in slut:
    minn = int(time[-9:-7])
    hour = int(time[-12:-10])
    minutes = hour*60 + minn
    stop_times.append(minutes)

times = []
for i in range(len(start_times)):
    #print(stop_times[i], start_times[i], stop_times[i] - start_times[i])
    times.append(stop_times[i] - start_times[i])
    
# Add total time to dataframe
combined_df['time'] = times

# Remove all fetuers we don't want
features_df = combined_df.drop(["Der_Behandling_PK", 
                               "Der_Opkort_FK",
                               "Der_Anestesikort_FK",
                               "BehandlingsStatus",
                               "ForberedelsetidStartTidpunkt",
                               "ForberedelsetidSlutTidpunkt",
                               "Primär_Sekundär_x",
                               "Primär_Sekundär_y",
                            ], axis='columns')

ingrepp_plural = {}
ingreppsgrupp = []
for index, row in combined_df.iterrows():
    ingrepp = row['Ingreppkod']
    ingrepp_group = ingrepp[0:2]
    ingreppsgrupp.append(ingrepp_group)
features_df['IngreppsGrupp'] = ingreppsgrupp

diagnosgrupp = []
for index, row in combined_df.iterrows():
    diagnos = row['Diagnoskod']
    diagnos_grupp = diagnos[0]
    diagnosgrupp.append(diagnos_grupp)
features_df['DiagnosGrupp'] = diagnosgrupp

'''
diagnosgrupper = {}
for diagnosgrupp, diagnosgrupp_df in features_df.groupby('DiagnosGrupp'):
    diagnosgrupper[diagnosgrupp] = diagnosgrupp_df
grupp_mean = []
grupp_std = []
for grupp in diagnosgrupper.keys():
    df = features_df[features_df['DiagnosGrupp'] == grupp]
    grupp_mean.append(df['time'].mean())
    grupp_std.append(df['time'].std())
#plt.errorbar(diagnosgrupper.keys(), grupp_mean, grupp_std, marker='o', linestyle='None', capsize=3)

ingreppsgrupper = {}
for ingreppsgrupp, ingreppsgrupp_df in features_df.groupby('IngreppsGrupp'):
    ingreppsgrupper[ingreppsgrupp] = ingreppsgrupp_df
grupp_mean = []
grupp_std = []
for grupp in ingreppsgrupper.keys():
    df = features_df[features_df['IngreppsGrupp'] == grupp]
    grupp_mean.append(df['time'].mean())
    grupp_std.append(df['time'].std())
#plt.errorbar(ingreppsgrupper.keys(), grupp_mean, grupp_std, marker='o', linestyle='None', capsize=3)
'''
features_df = features_df.drop(["Diagnoskod", "Ingreppkod", 'Kroppslängd', 'Kroppsvikt', 'Veckodag'], axis='columns')
features_df = features_df[features_df['IngreppsGrupp'].isin(['TN', 'NC', 'NH', 'NB', 'NG', 'NF', 'ND'])]

# Instansiate Metrics so we can use MAAPE later
metrics = Metrics()

C:\Users\gidja1\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (80,81) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Behandling length: 4581
Ingrepp length: 4757
Diagnos length: 4582
Combined length: 4563


In [57]:
features_df.head()

,Der_Vårdform_FK,Der_Prioritet_FK,ASAklass,PatientÅlderVidOp,Starttimme,BMI,Sida,time,IngreppsGrupp,DiagnosGrupp
0,2,2,NaN,29434.0,8.0,NaN,Höger,116,TN,T
1,2,3,NaN,16841.0,7.0,NaN,Höger,29,TN,M
2,2,1,3.0,15658.0,16.0,24.4,Vänster,88,NC,S
3,2,5,1.0,27854.0,11.0,29.1,Vänster,46,NH,T
4,2,6,NaN,21777.0,10.0,24.4,Vänster,95,NC,M


### Handle NaN (ONLY DO ONE OF THESE)

In [58]:
y = features_df["time"]
X = features_df.drop("time", axis='columns')

**Remove rows with NaN** (Good)

In [59]:
features_df = features_df.dropna()
print(len(features_df))
y = features_df["time"]
X = features_df.drop("time", axis='columns')

3708


**Replace NaN with 0** (Bad)

In [ ]:
X = X.fillna(0)
X.head()

**Simple Imputer** (Using most frequent)

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp.fit(X)
X = imp.transform(X)
X = pd.DataFrame(X).transpose()
X.head()

In [ ]:
imp = KNNImputer(missing_values=np.nan, n_neighbors=5)
imp.fit(X)
X = imp.transform(X)

### Encoding (ONLY DO ONE OF THESE)

**Use One Hot Encoding to encode "sida" and "ingreppsgrupp"** (This seems to be the better alternative)

In [60]:
X = pd.get_dummies(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=66)

**Ordinal Encoding**

In [21]:
#y = features_df['time']
#X = features_df.drop('time', axis='columns')
enc = OrdinalEncoder()
enc.fit(X)
X = enc.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=66)

### Trying models

In [61]:
# Try dummyregressor as a basecase, anything worse than this is really terrible
regr = DummyRegressor()
regr.fit(X_train, y_train)
pred = regr.predict(X_test)
abs_error = mean_squared_error(y_test, pred, squared=False)
percentage_error = metrics.mean_arctangent_absolute_percentage_error(clean=True, y_pred=np.array(pred), y_true=np.array(y_test))
print(f'abs error:{abs_error}, % error: {percentage_error}')

abs error:38.29162641919161, % error: 0.34


In [62]:
regr = DecisionTreeRegressor()
regr.fit(X_train, y_train)
pred = regr.predict(X_test)
abs_error = mean_squared_error(y_test, pred, squared=False)
percentage_error = metrics.mean_arctangent_absolute_percentage_error(clean=True, y_pred=np.array(pred), y_true=np.array(y_test))
print(f'abs error:{abs_error}, % error: {percentage_error}')

abs error:46.75895866836355, % error: 0.331


In [85]:
forest_regr = RandomForestRegressor(max_depth=22)
forest_regr.fit(X_train, y_train)
pred = forest_regr.predict(X_test)
abs_error = mean_squared_error(y_test, pred, squared=False)
percentage_error = metrics.mean_arctangent_absolute_percentage_error(clean=True, y_pred=np.array(pred), y_true=np.array(y_test))
print(f'abs error:{abs_error}, % error: {percentage_error}')

abs error:33.32669863224349, % error: 0.272


In [86]:
boost_regr = GradientBoostingRegressor(max_depth=3,)
boost_regr.fit(X_train, y_train)
pred = boost_regr.predict(X_test)
abs_error = mean_squared_error(y_test, pred, squared=False)
percentage_error = metrics.mean_arctangent_absolute_percentage_error(clean=True, y_pred=np.array(pred), y_true=np.array(y_test))
print(f'abs error:{abs_error}, % error: {percentage_error}')

abs error:31.76912603883179, % error: 0.265


In [65]:
regr = MLPRegressor(random_state=1, activation='logistic', learning_rate='adaptive', )
regr.fit(X_train, y_train)
pred = regr.predict(X_test)
abs_error = mean_squared_error(y_test, pred, squared=False)
percentage_error = metrics.mean_arctangent_absolute_percentage_error(clean=True, y_pred=np.array(pred), y_true=np.array(y_test))
print(f'abs error:{abs_error}, % error: {percentage_error}')

abs error:38.29116238181593, % error: 0.34


C:\Users\gidja1\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [79]:
regr = LinearRegression()
regr.fit(X_train, y_train)
pred = regr.predict(X_test)
abs_error = mean_squared_error(y_test, pred, squared=False)
percentage_error = metrics.mean_arctangent_absolute_percentage_error(clean=True, y_pred=np.array(pred), y_true=np.array(y_test))
print(f'abs error:{abs_error}, % error: {percentage_error}')

abs error:34.69773020618116, % error: 0.301


## Keras

In [69]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(31, input_dim=31, kernel_initializer='normal', activation='relu')) #Input layer
    model.add(Dense(10, kernel_initializer='normal', activation='relu')) #Input layer
    model.add(Dense(1, kernel_initializer='normal', activation='relu')) #Output layer
    # Compile model
    opt = keras.optimizers.Adam(clipnorm=1, learning_rate=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model
estimator = KerasRegressor(build_fn=baseline_model, epochs=1000, batch_size=32, verbose=1)
kfold = KFold(n_splits=10)
#results = cross_val_score(estimator, X_train, y_train, cv=kfold)


In [17]:
avrg = sum(results)/len(results)
print(f'avrg mean_error_sqr: {avrg}')

avrg mean_error_sqr: -6808.340002441406


In [67]:
estimator.fit(X_train, y_train)
pred = estimator.predict(X_test)

Epoch 1/1000
87/87 [==============================] - 1s 2ms/step - loss: 10338.9371
Epoch 2/1000
87/87 [==============================] - 0s 1ms/step - loss: 10005.2634
Epoch 3/1000
87/87 [==============================] - 0s 2ms/step - loss: 10150.5349
Epoch 4/1000
87/87 [==============================] - 0s 2ms/step - loss: 10621.6878
Epoch 5/1000
87/87 [==============================] - 0s 2ms/step - loss: 10407.3435
Epoch 6/1000
87/87 [==============================] - 0s 2ms/step - loss: 10117.5576
Epoch 7/1000
87/87 [==============================] - 0s 2ms/step - loss: 10102.3414
Epoch 8/1000
87/87 [==============================] - 0s 2ms/step - loss: 10053.7564
Epoch 9/1000
87/87 [==============================] - 0s 2ms/step - loss: 10526.1427
Epoch 10/1000
87/87 [==============================] - 0s 2ms/step - loss: 10268.3110
Epoch 11/1000
87/87 [==============================] - 0s 2ms/step - loss: 10193.9148
Epoch 12/1000
87/87 [==============================] - 0s 2ms/s

KeyboardInterrupt: 

In [87]:
importance = forest_regr.feature_importances_